<a href="https://colab.research.google.com/github/valeria-marquez-pellegrin/20.440-HW6/blob/main/Formal_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install biomart
!pip install fastcluster
!pip install gseapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.0/194.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.4 MB/s eta 0:00:00


### Libraries

In [3]:
# Libraries
import seaborn as sns
import pandas as pd
import scipy
import numpy as np
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
import matplotlib.colors
from matplotlib.lines import Line2D
import gseapy as gp
import networkx as nx


from scipy.spatial.distance import pdist, squareform
from sklearn.preprocessing import StandardScaler

from biomart import BiomartServer
from io import StringIO
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import silhouette_score
from sklearn.tree import export_graphviz

import graphviz
import statsmodels.api as sm

### Drive access

In [4]:
# Mount Google Drive. Permit access when prompted.
from google.colab import drive
from google.colab import files
drive.mount('/content/google_drive')

import os
os.chdir('/content/google_drive/MyDrive/')

Mounted at /content/google_drive


### Upload data frames

In [5]:
# Check directory for file names
directory_files = os.listdir('/content/google_drive/MyDrive/440 project E + V 4ever/Data')
for file in directory_files:
  print(file)

original_df = pd.read_csv('440 project E + V 4ever/Data/All_data.txt', sep = '\t')


All_data.txt
Old data
Patient baseline
Controls
Patient wk12


### Cleaning data

In [6]:
# Clean NA from dataset
original_df.dropna()
original_df

# Notes on original_df:
# Patients 5, 7, and 12 are missing because they didn't receive biopsies at week 12
# Patients 5, 6, and 7 are non-responders; thus, original_df includes data from a non-responder (patient 6)

,ID_REF,C1,C2,C3,P1_0,P2_0,P3_0,P4_0,P6_0,P8_0,...,P11_0,P1_12,P2_12,P3_12,P4_12,P6_12,P8_12,P9_12,P10_12,P11_12
0,1007_s_at,10.5215,10.8274,10.3693,10.5765,10.8312,10.5242,10.5482,10.2570,10.5092,...,10.4264,10.5843,10.6156,10.4015,10.7438,10.1759,10.2026,10.5832,10.6590,10.3895
1,1053_at,6.8552,6.7757,6.3699,6.2369,6.7993,6.9653,6.8716,6.7159,6.5620,...,6.5778,6.6646,7.0293,6.8159,7.3455,6.5795,6.5322,7.2529,7.0895,6.3157
2,117_at,3.7309,3.9160,4.1976,6.5442,5.9268,5.2193,4.5238,4.1740,4.7661,...,5.7193,5.0663,5.0403,4.7337,4.6863,4.3216,4.1662,4.6153,4.2891,5.1490
3,121_at,3.6326,4.5037,3.1192,3.1939,4.3667,3.3964,3.2283,3.4405,3.7721,...,3.7237,3.2672,3.9808,3.2856,3.6253,3.7397,3.7086,3.6810,4.3130,3.7313
4,1255_g_at,2.3670,2.3441,2.3478,2.5847,2.1721,2.5007,2.3445,2.3325,2.3950,...,2.3518,2.4053,2.4576,2.4955,2.4167,2.5382,2.4897,2.5572,2.4021,2.7009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54670,AFFX-ThrX-5_at,2.7630,2.6666,2.5998,2.7451,2.6632,2.5800,2.6701,2.7759,2.6888,...,2.7511,2.6842,2.7534,2.7975,2.5118,2.7033,2.6762,2.6209,2.6997,2.9352
54671,AFFX-ThrX-M_at,2.0498,2.2161,2.2586,2.1532,2.0683,2.0817,2.1490,2.0124,2.1615,...,2.0519,2.1507,2.1778,2.2616,2.1413,2.1011,2.1724,2.2977,2.0997,2.2687
54672,AFFX-TrpnX-3_at,2.0702,2.0976,1.9971,1.9826,2.0452,1.9457,2.0245,1.9737,2.0037,...,2.1885,2.1509,2.0023,1.8615,2.0019,2.0099,1.8929,1.9471,1.9811,2.3038
54673,AFFX-TrpnX-5_at,2.3267,2.3093,2.3798,2.2901,2.3076,2.3861,2.4244,2.3569,2.2092,...,2.2751,2.3538,2.4310,2.2822,2.2464,2.2731,2.3211,2.2894,2.3931,2.6518


In [7]:
# This df only includes controls and responders - drop the non-responder P6
# Controls = columns 1-3 (n = 3)
# Patients t0 = columns 4-11 (n = 8)
# Patients t12 = 12-19 (n = 8)

df_responders = original_df.drop(columns=['P6_0', 'P6_12'])
df_responders

,ID_REF,C1,C2,C3,P1_0,P2_0,P3_0,P4_0,P8_0,P9_0,P10_0,P11_0,P1_12,P2_12,P3_12,P4_12,P8_12,P9_12,P10_12,P11_12
0,1007_s_at,10.5215,10.8274,10.3693,10.5765,10.8312,10.5242,10.5482,10.5092,10.0938,10.9853,10.4264,10.5843,10.6156,10.4015,10.7438,10.2026,10.5832,10.6590,10.3895
1,1053_at,6.8552,6.7757,6.3699,6.2369,6.7993,6.9653,6.8716,6.5620,7.0051,6.4939,6.5778,6.6646,7.0293,6.8159,7.3455,6.5322,7.2529,7.0895,6.3157
2,117_at,3.7309,3.9160,4.1976,6.5442,5.9268,5.2193,4.5238,4.7661,4.9087,4.4371,5.7193,5.0663,5.0403,4.7337,4.6863,4.1662,4.6153,4.2891,5.1490
3,121_at,3.6326,4.5037,3.1192,3.1939,4.3667,3.3964,3.2283,3.7721,4.0848,3.8689,3.7237,3.2672,3.9808,3.2856,3.6253,3.7086,3.6810,4.3130,3.7313
4,1255_g_at,2.3670,2.3441,2.3478,2.5847,2.1721,2.5007,2.3445,2.3950,2.3723,2.2990,2.3518,2.4053,2.4576,2.4955,2.4167,2.4897,2.5572,2.4021,2.7009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54670,AFFX-ThrX-5_at,2.7630,2.6666,2.5998,2.7451,2.6632,2.5800,2.6701,2.6888,2.7270,2.7959,2.7511,2.6842,2.7534,2.7975,2.5118,2.6762,2.6209,2.6997,2.9352
54671,AFFX-ThrX-M_at,2.0498,2.2161,2.2586,2.1532,2.0683,2.0817,2.1490,2.1615,2.1578,2.1847,2.0519,2.1507,2.1778,2.2616,2.1413,2.1724,2.2977,2.0997,2.2687
54672,AFFX-TrpnX-3_at,2.0702,2.0976,1.9971,1.9826,2.0452,1.9457,2.0245,2.0037,2.0729,1.9118,2.1885,2.1509,2.0023,1.8615,2.0019,1.8929,1.9471,1.9811,2.3038
54673,AFFX-TrpnX-5_at,2.3267,2.3093,2.3798,2.2901,2.3076,2.3861,2.4244,2.2092,2.3552,2.2898,2.2751,2.3538,2.4310,2.2822,2.2464,2.3211,2.2894,2.3931,2.6518


In [8]:
# Groups

controls = pd.DataFrame(columns = ['C0', 'C1', 'C2'])
controls['C0'] = df_responders.iloc[:,1]
controls['C1'] = df_responders.iloc[:,2]
controls['C2'] = df_responders.iloc[:,3]

p0 = pd.DataFrame(columns = ['P1', 'P2', 'P3','P4', 'P8', 'P9','P10', 'P11'])
p0['P1'] = df_responders.iloc[:,4]
p0['P2'] = df_responders.iloc[:,5]
p0['P3'] = df_responders.iloc[:,6]
p0['P4'] = df_responders.iloc[:,7]
p0['P8'] = df_responders.iloc[:,8]
p0['P9'] = df_responders.iloc[:,9]
p0['P10'] = df_responders.iloc[:,10]
p0['P11'] = df_responders.iloc[:,11]

p12 = pd.DataFrame(columns = ['P1', 'P2', 'P3','P4', 'P8', 'P9','P10', 'P11'])
p12['P1'] = df_responders.iloc[:,12]
p12['P2'] = df_responders.iloc[:,13]
p12['P3'] = df_responders.iloc[:,14]
p12['P4'] = df_responders.iloc[:,15]
p12['P8'] = df_responders.iloc[:,16]
p12['P9'] = df_responders.iloc[:,17]
p12['P10'] = df_responders.iloc[:,18]
p12['P11'] = df_responders.iloc[:,19]

### Shapiro Wilk (normality)

In [ ]:
# Test each group (controls, P_0, P_12) for normality

normal_results = []
for a in range(len(df_responders)):
  s_test_c, s_p_c = scipy.stats.shapiro(controls.iloc[a,:])

  s_test_p0, s_p_p0 = scipy.stats.shapiro(p0.iloc[a,:])

  s_test_p12, s_p_p12 = scipy.stats.shapiro(p12.iloc[a,:])

  normal_results.append({
    'ID_REF': df_responders.iloc[a, 0],
    'C_pvalue': s_p_c,
    'P0_pvalue': s_p_p0,
    'P12_pvalue': s_p_p12
  })

shapiro_results = pd.DataFrame(normal_results)
shapiro_results

/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:1879: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "
/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:1887: RuntimeWarning: invalid value encountered in arccos
  pw = 1 - 6/np.pi*np.arccos(np.sqrt(w))


### Mann Whitney

In [ ]:
# There are a large number of non-normal distributions in our data, so use non-parametric statistics for DEG analysis
# Mann-Whitney U test

mann_results = []
for a in range(len(df_responders)):
  c_p0_test, c_p0_p = scipy.stats.mannwhitneyu(controls.iloc[a,:],p0.iloc[a,:])

  log2fc_1 = np.mean(p0.iloc[a,:])-np.mean(controls.iloc[a,:])

  c_p12_test, c_p12_p = scipy.stats.mannwhitneyu(controls.iloc[a,:],p12.iloc[a,:])

  log2fc_2 = np.mean(p12.iloc[a,:])-np.mean(controls.iloc[a,:])

  p0_p12_test, p0_p12_p = scipy.stats.mannwhitneyu(p0.iloc[a,:],p12.iloc[a,:])

  log2fc_3 = np.mean(p12.iloc[a,:])-np.mean(p0.iloc[a,:])

  mann_results.append({
    'ID_REF': df_responders.iloc[a, 0],
    'C vs P0 pvalue': c_p0_p,
    'C vs P0 log2fc': log2fc_1,
    'C vs P12 pvalue': c_p12_p,
    'C vs P12 log2fc': log2fc_2,
    'P0 vs P12 pvalue': p0_p12_p,
    'P0 vs P12 log2fc': log2fc_3
  })

mann_whitney_results = pd.DataFrame(mann_results)
mann_whitney_results

### T Test

In [ ]:
# Student's T test

t_results = []
for a in range(len(df_responders)):
  c_p0_test, c_p0_p = scipy.stats.ttest_ind(controls.iloc[a,:],p0.iloc[a,:])

  log2fc_1 = np.mean(p0.iloc[a,:])-np.mean(controls.iloc[a,:])

  c_p12_test, c_p12_p = scipy.stats.ttest_ind(controls.iloc[a,:],p12.iloc[a,:])

  log2fc_2 = np.mean(p12.iloc[a,:])-np.mean(controls.iloc[a,:])

  p0_p12_test, p0_p12_p = scipy.stats.ttest_ind(p0.iloc[a,:],p12.iloc[a,:])

  log2fc_3 = np.mean(p12.iloc[a,:])-np.mean(p0.iloc[a,:])

  t_results.append({
    'ID_REF': df_responders.iloc[a, 0],
    'C vs P0 pvalue': c_p0_p,
    'C vs P0 log2fc': log2fc_1,
    'C vs P12 pvalue': c_p12_p,
    'C vs P12 log2fc': log2fc_2,
    'P0 vs P12 pvalue': p0_p12_p,
    'P0 vs P12 log2fc': log2fc_3
  })

t_test_results = pd.DataFrame(t_results)
t_test_results

In [ ]:
# Make tables of the significant genes for each comparison for the supplement
CvsP0 = t_test_results.loc[(t_test_results['C vs P0 pvalue'] < 0.05) & (abs(t_test_results['C vs P0 log2fc'] > 1))]
CvsP12 = t_test_results.loc[(t_test_results['C vs P12 pvalue'] < 0.05) & (abs(t_test_results['C vs P12 log2fc'] > 1))]
P0vsP12 = t_test_results.loc[(t_test_results['P0 vs P12 pvalue'] < 0.05) & (abs(t_test_results['P0 vs P12 log2fc'] > 1))]

In [ ]:
# Save csv files to our drive folder
CvsP0.drop(columns = ['C vs P12 pvalue','C vs P12 log2fc', 'P0 vs P12 pvalue', 'P0 vs P12 log2fc']).to_csv('/content/google_drive/MyDrive/440 project E + V 4ever/CvsP0.csv')
CvsP12.drop(columns = ['C vs P0 pvalue','C vs P0 log2fc', 'P0 vs P12 pvalue', 'P0 vs P12 log2fc']).to_csv('/content/google_drive/MyDrive/440 project E + V 4ever/CvsP12.csv')
P0vsP12.drop(columns = ['C vs P12 pvalue','C vs P12 log2fc', 'C vs P0 pvalue', 'C vs P0 log2fc']).to_csv('/content/google_drive/MyDrive/440 project E + V 4ever/P0vsP12.csv')

### Enrichment analysis

In [ ]:
unique_affy_ids = t_test_results['ID_REF'].unique()  # Ensure unique IDs

# Connect to the BioMart server
server = BiomartServer("http://www.ensembl.org/biomart")

# Select the appropriate dataset; for human genes, use 'hsapiens_gene_ensembl'
dataset = server.datasets['hsapiens_gene_ensembl']

# Perform the search
# Adjust 'attributes' and 'filters' accordingly
# Affymetrix array used: 'affy_hg_u133_plus_2'
response = dataset.search({
    'filters': {
        'with_affy_hg_u133_plus_2': True  # Get all genes with Affy HG U133 Plus 2.0
        },
    'attributes': [
        'affy_hg_u133_plus_2',  # Affymetrix ID
        'ensembl_gene_id',     # Ensembl Gene ID
        'external_gene_name'   # Gene name
    ]
})

# Convert the response text to a DataFrame
hsapiens_ensembl = pd.read_csv(StringIO(response.text), sep='\t', names=['ID_REF', 'Ensembl_ID', 'Gene_Name'])

# Merge to find matches
matched_data_filtered = hsapiens_ensembl[hsapiens_ensembl['ID_REF'].isin(unique_affy_ids)]

In [ ]:
# Merge t_test_results with gene names for annotation
t_test_results_with_genes = t_test_results.merge(matched_data_filtered, on='ID_REF', how='left')

CvsP0 = t_test_results_with_genes.loc[(t_test_results_with_genes['C vs P0 pvalue'] < 0.05) & (abs(t_test_results_with_genes['C vs P0 log2fc'] > 1))]
CvsP12 = t_test_results_with_genes.loc[(t_test_results_with_genes['C vs P12 pvalue'] < 0.05) & (abs(t_test_results_with_genes['C vs P12 log2fc'] > 1))]
P0vsP12 = t_test_results_with_genes.loc[(t_test_results_with_genes['P0 vs P12 pvalue'] < 0.05) & (abs(t_test_results_with_genes['P0 vs P12 log2fc'] > 1))]

In [ ]:
# Save csv files to our drive folder
CvsP0.drop(columns = ['C vs P12 pvalue','C vs P12 log2fc', 'P0 vs P12 pvalue', 'P0 vs P12 log2fc']).to_csv('/content/google_drive/MyDrive/440 project E + V 4ever/Results/CvsP0.csv')
CvsP12.drop(columns = ['C vs P0 pvalue','C vs P0 log2fc', 'P0 vs P12 pvalue', 'P0 vs P12 log2fc']).to_csv('/content/google_drive/MyDrive/440 project E + V 4ever/Results/CvsP12.csv')
P0vsP12.drop(columns = ['C vs P12 pvalue','C vs P12 log2fc', 'C vs P0 pvalue', 'C vs P0 log2fc']).to_csv('/content/google_drive/MyDrive/440 project E + V 4ever/Results/P0vsP12.csv')

### Volcano plot

In [ ]:
# Calculate -log10(p-value) for the y-axis for all comparisons
t_test_results_with_genes['neg_log10_C vs P0 pvalue'] = -np.log10(t_test_results_with_genes['C vs P0 pvalue'])
t_test_results_with_genes['neg_log10_C vs P12 pvalue'] = -np.log10(t_test_results_with_genes['C vs P12 pvalue'])
t_test_results_with_genes['neg_log10_P0 vs P12 pvalue'] = -np.log10(t_test_results_with_genes['P0 vs P12 pvalue'])

# Define thresholds for significance
logfc_threshold = 1  # log fold change threshold
pvalue_threshold = 0.05  # p-value threshold

# Initialize dictionaries to store significant genes for each comparison
significant_genes = {
    'C vs P0': {'upregulated': None, 'downregulated': None},
    'C vs P12': {'upregulated': None, 'downregulated': None},
    'P0 vs P12': {'upregulated': None, 'downregulated': None}
}

# Create subplots for each comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Titles for the subplots
titles = ['C vs P0', 'C vs P12', 'P0 vs P12']


# Custom legend handles
legend_elements = [
    Line2D([0], [0], marker='o', color='w', label='Not significant',
           markerfacecolor='grey', markersize=10),
    Line2D([0], [0], marker='o', color='w', label='Upregulated',
           markerfacecolor='red', markersize=10),
    Line2D([0], [0], marker='o', color='w', label='Downregulated',
           markerfacecolor='blue', markersize=10),
]

# Iterate over the comparisons to create each subplot and filter/store significant genes
for i, (logfc_col, pval_col, title) in enumerate(zip(
        ['C vs P0 log2fc', 'C vs P12 log2fc', 'P0 vs P12 log2fc'],
        ['neg_log10_C vs P0 pvalue', 'neg_log10_C vs P12 pvalue', 'neg_log10_P0 vs P12 pvalue'],
        titles)):

    significant_up = t_test_results_with_genes[
        (t_test_results_with_genes[logfc_col] >= logfc_threshold) &
        (t_test_results_with_genes[pval_col.replace('neg_log10_', '')] <= pvalue_threshold)
    ]

    significant_down = t_test_results_with_genes[
        (t_test_results_with_genes[logfc_col] <= -logfc_threshold) &
        (t_test_results_with_genes[pval_col.replace('neg_log10_', '')] <= pvalue_threshold)
    ]

    non_significant = t_test_results_with_genes[
        (abs(t_test_results_with_genes[logfc_col]) < logfc_threshold) |
        (t_test_results_with_genes[pval_col.replace('neg_log10_', '')] > pvalue_threshold)
    ]

    # Store in dictionary
    significant_genes[title]['upregulated'] = significant_up
    significant_genes[title]['downregulated'] = significant_down

    # Scatter plot of non-significant points
    axes[i].scatter(
        x=non_significant[logfc_col],
        y=non_significant[pval_col],
        s=10,  # smaller marker size for non-significant points
        alpha=0.5,  # slightly transparent
        color='grey',  # grey color for non-significant points
        edgecolors='none'  # no edge color to markers
    )

    # Scatter plot of significant upregulated points
    axes[i].scatter(
        x=significant_up[logfc_col],
        y=significant_up[pval_col],
        s=50,  # larger marker size for significant points
        color='red',  # red color for significant upregulated points
        edgecolors='none'
    )

    # Scatter plot of significant downregulated points
    axes[i].scatter(
        x=significant_down[logfc_col],
        y=significant_down[pval_col],
        s=50,  # larger marker size for significant points
        color='blue',  # blue color for significant downregulated points
        edgecolors='none'
    )

    # For upregulated genes, this means the largest positive fold change
    top_genes_up = significant_up.sort_values(
        by=[pval_col.replace('neg_log10_', ''), logfc_col], ascending=[True, False]
        ).head(5)

    # For downregulated genes, this means the largest negative fold change
    top_genes_down = significant_down.sort_values(
        by=[pval_col.replace('neg_log10_', ''), logfc_col], ascending=[True, True]
        ).head(5)

    # Select top genes by smallest p-value for upregulated and downregulated
    #top_genes_up = significant_up.nsmallest(5, pval_col.replace('neg_log10_', ''))
    #top_genes_down = significant_down.nsmallest(5, pval_col.replace('neg_log10_', ''))

    # Combine top upregulated and downregulated genes
    top_genes = pd.concat([top_genes_up, top_genes_down])

    # Annotate top genes
    for _, row in top_genes.iterrows():
        ha = 'center'
        va = 'bottom' if row[logfc_col] > 0 else 'top'
        axes[i].text(
            row[logfc_col],
            row[pval_col],
            row['Gene_Name'],
            ha=ha,
            va=va,
            fontsize=9,  # Adjust fontsize according to your plot size
            color='darkgreen',  # Choose a color that is visible
            weight='bold',  # Bold text to make it stand out
            clip_on=True,  # Prevent clipping at edges
        )
    # Adding the legend to each subplot
    axes[i].legend(handles=legend_elements, loc='upper right')

    # Enhancing each subplot
    axes[i].set_title(f'Volcano Plot of {title}')
    axes[i].set_xlabel('Log2 Fold Change')
    axes[i].set_ylabel('-Log10(p-value)')
    axes[i].axhline(y=-np.log10(pvalue_threshold), color='black', linestyle='--', linewidth=1)
    axes[i].axvline(x=logfc_threshold, color='black', linestyle='--', linewidth=1)
    axes[i].axvline(x=-logfc_threshold, color='black', linestyle='--', linewidth=1)
    axes[i].grid(True, linestyle=':', color='grey')  # Add a dotted grid for better readability

    plt.savefig(f'/content/google_drive/MyDrive/440 project E + V 4ever/Results/Volcano Plot of {title}.jpeg')

plt.tight_layout()
plt.show()

In [ ]:
# Dictionary to store the cleaned DataFrames
cleaned_data_frames = {}

# Iterate through the significant_genes to create, merge, and clean DataFrames
for group, data in significant_genes.items():
    # Combine upregulated and downregulated genes into one DataFrame with a 'Regulation' column
    upregulated_df = data['upregulated'].copy()
    upregulated_df['Regulation'] = 'upregulated'
    downregulated_df = data['downregulated'].copy()
    downregulated_df['Regulation'] = 'downregulated'
    combined_df = pd.concat([upregulated_df, downregulated_df])

    # Select specific columns and merge with df_responders
    significant_all_data = combined_df[['ID_REF', 'Ensembl_ID', 'Gene_Name', 'Regulation']].merge(df_responders, on='ID_REF', how='left')

    # Drop rows with any NaN values
    cleaned_df = significant_all_data.dropna()

    # Store the cleaned DataFrame in the dictionary
    cleaned_data_frames[group] = cleaned_df

# Now, cleaned_data_frames contains cleaned DataFrames for each group
# Accessing the cleaned DataFrames, for example:
C_vs_P0_cleaned = cleaned_data_frames['C vs P0']
C_vs_P12_cleaned = cleaned_data_frames['C vs P12']
P0_vs_P12_cleaned = cleaned_data_frames['P0 vs P12']

### Heatmap

In [ ]:
# List of cleaned DataFrames
cleaned_data_frames = [C_vs_P0_cleaned, C_vs_P12_cleaned, P0_vs_P12_cleaned]
comparison_names = ['C vs P0', 'C vs P12', 'P0 vs P12']

for df, name in zip(cleaned_data_frames, comparison_names):

    # Adapting the dataframe for the heatmap
    heatmap_data = df.set_index('Gene_Name')
    heatmap_data_plot = heatmap_data.iloc[:, 3:]

    # Create the clustermap
    g = sns.clustermap(heatmap_data_plot, method='average', cmap='viridis', standard_scale=1,
                       xticklabels=True, yticklabels=True)

    plt.setp(g.ax_heatmap.get_yticklabels(), fontsize=1)  # Set fontsize to a readable size

    # Set the color bar to the new position
    pos = g.cax.get_position()
    g.cax.set_position([pos.x0 - 0.15, pos.y0, pos.width, pos.height])  # Adjust if necessary

    plt.title(f'Gene Expression Heatmap - {name}')
    plt.show()


In [ ]:
# Testing a new standardization method: z score instead of seaborn's method

# List of cleaned DataFrames
cleaned_data_frames = [C_vs_P0_cleaned, C_vs_P12_cleaned, P0_vs_P12_cleaned]
comparison_names = ['C vs P0', 'C vs P12', 'P0 vs P12']

for df, name in zip(cleaned_data_frames, comparison_names):

    # Adapting the dataframe for the heatmap
    heatmap_data = df.set_index('Gene_Name')
    heatmap_data_plot = heatmap_data.iloc[:, 3:]

    # Standardize values according using z-score
    means = []
    stds = []
    for a in range(len(heatmap_data_plot.iloc[0,:])):
      means.append(np.mean(heatmap_data_plot.iloc[:,a]))
      stds.append(np.std(heatmap_data_plot.iloc[:,a]))

    count = 0
    while count < len(heatmap_data_plot.iloc[0,:]):
      sub = 0
      while sub < len(heatmap_data_plot):
        heatmap_data_plot.iloc[sub,count] = (heatmap_data_plot.iloc[sub, count] - means[count])/stds[count]
        sub += 1
      count += 1

    # Create the clustermap
    g = sns.clustermap(heatmap_data_plot, method='average', cmap='viridis',
                       xticklabels=True, yticklabels=True)

    plt.setp(g.ax_heatmap.get_yticklabels(), fontsize=1)  # Set fontsize to a readable size

    # Set the color bar to the new position
    pos = g.cax.get_position()
    g.cax.set_position([pos.x0 - 0.15, pos.y0, pos.width, pos.height])  # Adjust if necessary

    plt.title(f'Gene Expression Heatmap - {name}')
    plt.savefig(f'/content/google_drive/MyDrive/440 project E + V 4ever/Results/Gene Expression Heatmap - {name}.jpeg')
    plt.show()

### KEGG

In [ ]:
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
def run_up_kegg_enrichment(df, comparison_name):
    upregulated_genes = df[df['Regulation'] == 'upregulated'].index.tolist()
    # Run KEGG enrichment analysis for upregulated genes
    up_kegg_enr = gp.enrichr(gene_list=upregulated_genes,
                             gene_sets='KEGG_2021_Human',
                             organism = 'human',
                             outdir=f'kegg_pathway_enrichment_results/{comparison_name}_upregulated',
                             no_plot=True)
    print(f"KEGG Enrichment Analysis (Upregulated) - {comparison_name}")
    print(up_kegg_enr.results.head())
    # Return the Enrichr object
    return up_kegg_enr

def run_down_kegg_enrichment(df, comparison_name):
    downregulated_genes = df[df['Regulation'] == 'downregulated'].index.tolist()
    # Run KEGG enrichment analysis for downregulated genes
    down_kegg_enr = gp.enrichr(gene_list=downregulated_genes,
                               gene_sets='KEGG_2021_Human',
                               organism = 'human',
                               outdir=f'kegg_pathway_enrichment_results/{comparison_name}_downregulated',
                               no_plot=True)
    # Return the Enrichr object

    return down_kegg_enr

def plot_kegg_enrichment_results(results_dict, regulation_type):
    for comparison_name, enrichr_obj in results_dict.items():
        # Check if the results DataFrame is empty
        if not enrichr_obj.results.empty:
            plt.figure(figsize=(3,5))
            # Create the dotplot
            gp.dotplot(enrichr_obj.res2d, title=f'{comparison_name} {regulation_type} KEGG Pathways',
                       cmap='viridis_r', y='Term', size=5)
            # Save the plot
            plt.savefig(f'kegg_dotplot_{comparison_name}_{regulation_type}.png')
            plt.show()  # Show the plot


data_frames_up = {'C vs P0': C_vs_P0_cleaned, 'C vs P12': C_vs_P12_cleaned, 'P0 vs P12': P0_vs_P12_cleaned}
data_frames_down = {'C vs P0': C_vs_P0_cleaned, 'P0 vs P12': P0_vs_P12_cleaned}
#C_vs_P12_cleaned was omitted for the down regulation since there was no enrich terms when cutoff = 0.05

# Assuming you have your data_frames_up and data_frames_down set up as before
up_kegg_results = {}
down_kegg_results = {}

for name, df in data_frames_up.items():
    up_kegg_results[name] = run_up_kegg_enrichment(df.set_index('Gene_Name'), name)

for name, df in data_frames_down.items():
    down_kegg_results[name] = run_down_kegg_enrichment(df.set_index('Gene_Name'), name)

# Now plot the results using dotplot
plot_kegg_enrichment_results(up_kegg_results, 'upregulated')
plot_kegg_enrichment_results(down_kegg_results, 'downregulated')


### GO

In [ ]:
def run_up_go_enrichment(df, comparison_name):
    upregulated_genes = df[df['Regulation'] == 'upregulated'].index.tolist()
    # Run KEGG enrichment analysis for upregulated genes
    up_go_enr = gp.enrichr(gene_list=upregulated_genes,
                             gene_sets='GO_Biological_Process_2021',
                             organism = 'human',
                             outdir=f'go_enrichment_results/{comparison_name}_upregulated',
                             no_plot=True)
    # Return the Enrichr object
    return up_go_enr

def run_down_go_enrichment(df, comparison_name):
    downregulated_genes = df[df['Regulation'] == 'downregulated'].index.tolist()
    # Run KEGG enrichment analysis for downregulated genes
    down_go_enr = gp.enrichr(gene_list=downregulated_genes,
                               gene_sets='GO_Biological_Process_2021',
                               organism = 'human',
                               outdir=f'go_enrichment_results/{comparison_name}_downregulated',
                               no_plot=True)
    # Return the Enrichr object
    return down_go_enr

def plot_go_enrichment_results(results_dict, regulation_type):
    for comparison_name, enrichr_obj in results_dict.items():
        # Check if the results DataFrame is empty
        if not enrichr_obj.results.empty:
            plt.figure(figsize=(3,5))
            # Create the dotplot
            gp.dotplot(enrichr_obj.res2d, title=f'{comparison_name} {regulation_type} GO Pathways',
                       cmap='viridis_r', y='Term', size=5)
            # Save the plot
            plt.savefig(f'go_dotplot_{comparison_name}_{regulation_type}.png')
            plt.show()  # Show the plot


data_frames_up = {'C vs P0': C_vs_P0_cleaned, 'C vs P12': C_vs_P12_cleaned, 'P0 vs P12': P0_vs_P12_cleaned}
data_frames_down = {'C vs P0': C_vs_P0_cleaned, 'P0 vs P12': P0_vs_P12_cleaned}
#C_vs_P12_cleaned was omitted for the down regulation since there was no enrich terms when cutoff = 0.05

# Assuming you have your data_frames_up and data_frames_down set up as before
up_go_results = {}
down_go_results = {}

for name, df in data_frames_up.items():
    up_go_results[name] = run_up_go_enrichment(df.set_index('Gene_Name'), name)

for name, df in data_frames_down.items():
    down_go_results[name] = run_down_go_enrichment(df.set_index('Gene_Name'), name)

# Now plot the results using dotplot
plot_go_enrichment_results(up_go_results, 'upregulated')
plot_go_enrichment_results(down_go_results, 'downregulated')


### Networks

In [ ]:

# Placeholder function to get gene interactions for a given pathway
# You will need to replace this with actual logic to get the data
def get_interactions_for_pathway(pathway_name):
    # This should return a list of tuples, where each tuple represents an interaction
    # Example: [('Gene1', 'Gene2'), ('Gene2', 'Gene3')]
    return []

# Function to draw a network for a given set of interactions
def draw_network(interactions, title):
    G = nx.Graph()
    G.add_edges_from(interactions)

    # Draw the network
    plt.figure(figsize=(8, 6))
    pos = nx.spring_layout(G)  # positions for all nodes
    nx.draw(G, pos, with_labels=True, node_color='skyblue', edge_color='gray', node_size=500)
    plt.title(title)
    plt.show()

# Function to plot networks for each KEGG pathway in the results
def plot_kegg_networks(kegg_results, regulation_type):
    for comparison_name, enrichr_obj in kegg_results.items():
        if not enrichr_obj.results.empty:
            # Iterate over each enriched pathway
            for index, row in enrichr_obj.results.iterrows():
                pathway_name = row['Term']
                interactions = get_interactions_for_pathway(pathway_name)
                if interactions:
                    draw_network(interactions, f'Network for {pathway_name} ({comparison_name} {regulation_type})')

# Example usage:
plot_kegg_networks(up_kegg_results, 'upregulated')
plot_kegg_networks(down_kegg_results, 'downregulated')


In [ ]:
!pip install bioservices

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from gseapy import enrichment_map

# Aggregate results from all comparisons into a single DataFrame
all_up_results = pd.concat([res.res2d for res in up_kegg_results.values()], ignore_index=True)
all_down_results = pd.concat([res.res2d for res in down_kegg_results.values()], ignore_index=True)

# Generate enrichment map for upregulated genes
nodes_up, edges_up = enrichment_map(all_up_results)

# Generate enrichment map for downregulated genes
nodes_down, edges_down = enrichment_map(all_down_results)

# Function to plot the network from nodes and edges
def plot_network(nodes, edges, title):
    # Build graph
    G = nx.from_pandas_edgelist(edges, source='src_idx', target='targ_idx', edge_attr=['jaccard_coef', 'overlap_coef', 'overlap_genes'])

    # Initialize node coordinates
    pos = nx.layout.spring_layout(G)  # spring layout tends to look better for these types of graphs

    # Draw network
    fig, ax = plt.subplots(figsize=(12, 12))
    nx.draw_networkx_nodes(G, pos, cmap=plt.cm.RdYlBu, node_color=list(nodes['NES']), node_size=list(nodes['Hits_ratio'] * 1000))
    nx.draw_networkx_labels(G, pos, labels=nodes['Term'].to_dict())
    edge_weight = nx.get_edge_attributes(G, 'jaccard_coef').values()
    nx.draw_networkx_edges(G, pos, width=list(map(lambda x: x * 10, edge_weight)), edge_color='#CDDBD4')

    plt.title(title)
    plt.show()

# Plot the networks
plot_network(nodes_up, edges_up, "Enrichment Map for Upregulated Genes")
plot_network(nodes_down, edges_down, "Enrichment Map for Downregulated Genes")


In [ ]:
from bioservices import KEGG

# Assuming up_kegg_results is populated from the previous enrichment analysis
# Here we use a simplified function to fetch interaction data

kegg = KEGG()

def get_pathway_interactions(pathway_id):
    # Fetch the detailed pathway data from KEGG
    pathway_data = kegg.get(pathway_id)
    pathway_info = kegg.parse(pathway_data)

    # Extract gene interactions from the pathway
    # This is simplified and would need to be adapted to the actual data structure
    interactions = []
    if 'RELATION' in pathway_info:
        for relation in pathway_info['RELATION']:
            if 'gene' in relation.keys():
                entry1 = relation['entry1']
                entry2 = relation['entry2']
                interactions.append((entry1, entry2))

    return interactions

def extract_and_plot_pathway_interactions(results_dict, regulation_type):
    for comparison_name, result in results_dict.items():
        # Iterate through each enriched pathway
        for index, row in result.res2d.iterrows():
            pathway_name = row['Term']
            pathway_id = row['Term ID'].split(':')[1]  # Assuming Term ID is in the format KEGG:pathway_id
            interactions = get_pathway_interactions(pathway_id)

            # Here you would call a function to plot these interactions
            # For now, just printing the pathway and the number of interactions
            print(f"{pathway_name} ({comparison_name} {regulation_type}): {len(interactions)} interactions")

# Example usage
extract_and_plot_pathway_interactions(up_kegg_results, 'upregulated')
